# This notebook pulls out financial data from finnhub only for the categories specified in the "Data Mapping" sheets on google Drive

In [109]:
import numpy as np
import pandas as pd
import json
import requests
import finnhub


## Giving one ticker and retrieving financial data

In [154]:
symbol = "?symbol="+ input()
token = "&token=c0j6hdv48v6tlon085cg"
r = requests.get('https://finnhub.io/api/v1/stock/profile2' + symbol + token)

r = r.json()

dataframe = pd.DataFrame.from_dict(r, orient="index")
dataframe

TSLA


,0
country,US
currency,USD
exchange,NASDAQ NMS - GLOBAL MARKET
finnhubIndustry,Automobiles
ipo,2010-06-29
logo,https://finnhub.io/api/logo?symbol=TSLA
marketCapitalization,689587.6
name,Tesla Inc
phone,16506815000.0
shareOutstanding,959.853504


In [6]:
dataframe.loc['name'][0]

'Tesla Inc'

# Giving a list of ticker symbols and retrieving relevant financial information

In [153]:
#API KEY
token = "&token=c0j6hdv48v6tlon085cg"



#List the tickers you want here


tickers = ['AAPL', 'TSLA', 'GOOGL']


#Getting the CompanyProfile2 endpoint from finnhub and taking certain ones out

responces1 = list()

for ticker in tickers:
    r1 = requests.get('https://finnhub.io/api/v1/stock/profile2?symbol={}'.format(ticker) + token)
    data1=json.loads(r1.text)
    responces1.append(data1)
    


df1 = pd.DataFrame.from_dict(responces1)
df1 = df1.drop(columns=['phone', 'logo'])
df1 = df1.rename(columns={'country':'Country', 'currency':'Currency', 'exchange':'Exchange', 'finnhubIndustry':'Industry', 'ipo':'IPO', 'marketCapitalization':'Market Capitalization', 'name':'Name', 'shareOutstanding':'Share Outstanding', 'ticker':'Ticker', 'weburl':'URL'})


#only select CompanyNewsScore from the sentiment score endpoint


responces3 = list()

for ticker in tickers:
    r3 = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol={}'.format(ticker) + token)
    data3=json.loads(r3.text)
    responces3.append(data3)
    


df3 = pd.DataFrame.from_dict(responces3)
df3 = pd.DataFrame(df3['companyNewsScore'])
df3 = df3.rename(columns={'companyNewsScore':'Company News Score'})


# select current stock price from the quote endpoint

responces4 = list()

for ticker in tickers:
    r4 = requests.get('https://finnhub.io/api/v1/quote?symbol={}'.format(ticker) + token)
    data4=json.loads(r4.text)
    responces4.append(data4)
    


df4 = pd.DataFrame.from_dict(responces4)
df4 = df4.drop(columns=['h', 'l', 'o', 'pc', 't'])
df4 = df4.rename(columns={'c':'Current Stock Price'})


#adding everything together, rearranging the columns

finaldf = pd.concat([df1, df3, df4], axis=1)
finaldf = finaldf[['Ticker', 'Name', 'Country', 'Currency', 'Exchange', 'Industry', 'IPO', 'Current Stock Price', 'Market Capitalization', 'Share Outstanding', 'Company News Score', 'URL']]
finaldf



,Ticker,Name,Country,Currency,Exchange,Industry,IPO,Current Stock Price,Market Capitalization,Share Outstanding,Company News Score,URL
0,AAPL,Apple Inc,US,USD,NASDAQ NMS - GLOBAL MARKET,Technology,1980-12-12,123.8350,2145351.0,16788.096000,0.6823,https://www.apple.com/
1,TSLA,Tesla Inc,US,USD,NASDAQ NMS - GLOBAL MARKET,Automobiles,2010-06-29,685.6699,689587.6,959.853504,0.3907,https://www.tesla.com/
2,GOOGL,Alphabet Inc,US,USD,NASDAQ NMS - GLOBAL MARKET,Media,2004-08-19,2041.6800,1399115.0,674.136665,0.3970,https://abc.xyz/
